<a href="https://colab.research.google.com/github/P3aar/Cartorioebac/blob/main/1_LimpezaDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from google.colab import files
uploaded = files.upload()


Saving avaliacoes_pedido.csv to avaliacoes_pedido.csv
Saving clientes.csv to clientes.csv
Saving itens_pedido.csv to itens_pedido.csv
Saving pagamentos_pedido.csv to pagamentos_pedido.csv
Saving pedidos.csv to pedidos.csv
Saving produtos.csv to produtos.csv
Saving vendedores.csv to vendedores.csv


In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
produtos = spark.read.csv('/content/produtos.csv', header=True, inferSchema=True)
vendedores = spark.read.csv('/content/vendedores.csv', header=True, inferSchema=True)
clientes = spark.read.csv('/content/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('/content/itens_pedido.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('/content/pagamentos_pedido.csv', header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv('/content/avaliacoes_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('/content/pedidos.csv', header=True, inferSchema=True)

In [13]:
print('dataframe produtos: \n', produtos.show(n=5, truncate=False))
print('dataframe vendedores: \n', vendedores.show(5))
print('dataframe clientes: \n', clientes.show(5))
print('dataframe itens_pedido: \n', itens_pedido.show(5))
print('dataframe pagamentos_pedido: \n', pagamentos_pedido.show(5))
print('dataframe avaliacoes_pedido: \n', avaliacoes_pedido.show(5))
print('dataframe pedidos: \n', pedidos.show(5))

+--------------------------------+---------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|id_produto                      |categoria_produto    |tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------------------+---------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff4541ed26657ea517e5|perfumaria           |40                  |287                      |1                       |225           |16                    |10               |14                |
|3aa071139cb16b67ca9e5dea641aaa2f|artes                |44                  |276                      |1                       |1000          |30                    |18               |

In [6]:
# Acessar colunas
from pyspark.sql.functions import col

clientes.select('id_cliente').show(1, truncate=False)
clientes.select(col('id_cliente')).show(1)
clientes.select(clientes['id_cliente']).show(1)
clientes.select(clientes.id_cliente).show(1)

+--------------------------------+
|id_cliente                      |
+--------------------------------+
|06b8999e2fba1a1fbc88172c00ba8bc7|
+--------------------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row

+--------------------+
|          id_cliente|
+--------------------+
|06b8999e2fba1a1fb...|
+--------------------+
only showing top 1 row



In [12]:
produtos_tratar_categ_nulos = produtos.na.fill({'categoria_produto': 'Nao especificado'})

produtos_tratar_categ_nulos.filter(col('categoria_produto') == 'Nao especificado').count()

610

In [14]:
print('Total de pedidos: ', pedidos.count())

pedidos_unicos = pedidos.dropDuplicates()
print('total de pedidos unicos: ', pedidos_unicos.count())

pedidos_remocao_nulos = pedidos_unicos.na.drop()
print('Total de pedidos apos limpeza de  valores nuloes: ', pedidos_remocao_nulos.count())

pedidos_remocao_nulos_id = pedidos_unicos.na.drop(subset=['id_cliente', 'id_pedido'])
print('Total de pedidos apos limpeza de id nulos: ', pedidos_remocao_nulos_id.count())

Total de pedidos:  99441
total de pedidos unicos:  99441
Total de pedidos apos limpeza de  valores nuloes:  96461
Total de pedidos apos limpeza de id nulos:  99441


In [15]:
colunas = ['peso_produto_g', 'comprimento_produto_cm', 'altura_produto_cm', 'largura_produto_cm']

for coluna in colunas:
  produtos = produtos.na.fill({coluna: 0})

In [17]:
produtos.write.mode('overwrite').option('header', 'true').csv('/content/drive/MyDrive/produto_tratada_csv')

In [18]:
spark.read.option('header', 'true').csv('/content/drive/MyDrive/produto_tratada_csv').show(5)

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
|96bd76ec8810374ed...|       esporte_lazer|  